In [ ]:
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# 生成随机点
def createUniformRandomDots(minVal, maxVal, count):
    dots = np.random.default_rng().uniform(minVal, maxVal, (count, 2))
    resdf = pd.DataFrame(dots, columns=["X", "Y"])
    plt.plot(resdf["X"], resdf["Y"], ".")
    plt.show()
    return resdf

In [ ]:
# 设置每点所对应的方块
def setBox(row, xy, boxSize, result):
    if (
        xy[0] < row["X"] < xy[0] + boxSize
        and xy[1] < row["Y"] < xy[1] + boxSize
        and len(row["BoxLoc"]) == 0
    ):
        row["BoxLoc"].append(xy[0])
        row["BoxLoc"].append(xy[1])
        result.append((xy[0], xy[1]))


# 计算所需方块
def calculateSurroundedBoxes(minVal, maxVal, dots, boxSize):
    dots["BoxLoc"] = dots.apply(lambda x: [], axis=1)
    result = []
    start = datetime.now()
    rowCount = len(dots)
    while True:
        xyArray = np.random.default_rng().uniform(minVal, maxVal, 2)
        dots.apply(lambda tmprow: setBox(tmprow, xyArray, boxSize, result), axis=1)
        if dots["BoxLoc"].map(lambda r: len(r)).sum() == rowCount * 2:
            break
    print("running time :", datetime.now() - start)
    return result

In [ ]:
samples = createUniformRandomDots(-20, 20, 20)

In [ ]:
calculateSurroundedBoxes(-20, 20, samples, 5)